In [4]:
from dataset import StateActionEffectDataset
train_set = StateActionEffectDataset("v4", "")

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
pybullet build time: May 20 2022 19:44:17


In [10]:
import wandb
run_id = "3okblzeq"
import utils
run = wandb.init(entity="colorslab",project="multideepsym", resume="must", id=run_id)
run = wandb.Api().run("colorslab/multideepsym/" + run_id)
model = utils.create_model_from_config(run.config)
model.load("_last", from_wandb=True)

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: mbatuhancelik (colorslab). Use `wandb login --relogin` to force relogin


wandb: ERROR Failed to sample metric: Not Supported


In [5]:
def get_obj_location(position):
    d = 0.9/8
    rel_x = position[0].item()- 0.5
    x_index = rel_x / d
    x_index = round(x_index)

    rel_y = position[1].item()- -0.5
    y_index = round(rel_y / d)

    x_index = min(x_index, 8- 1)
    y_index = min(y_index, 8 - 1)
    x_index = max(0, x_index)
    y_index = max(0, y_index)

    return [x_index, y_index]

In [2]:
import torch
cossim = torch.nn.CosineSimilarity(dim = -1)

In [6]:
state = train_set.state[0]
goal = train_set.state[1]

In [7]:
num_objects = torch.count_nonzero(torch.count_nonzero(goal, dim = -1))

In [8]:
num_objects

tensor(10)

cosine similarity between sequential states

In [9]:
torch.sum(cossim(state[:num_objects], goal[:num_objects]))

tensor(9.9988)

position changes on 3rd object

In [11]:
train_set.effect[100] > 1e-1

tensor([[False, False, False, False, False, False],
        [False, False, False, False, False, False],
        [False, False, False, False, False, False],
        [ True,  True, False, False, False, False],
        [False, False, False, False, False, False],
        [False, False, False, False, False, False],
        [False, False, False, False, False, False],
        [False, False, False, False, False, False],
        [False, False, False, False, False, False],
        [False, False, False, False, False, False],
        [False, False, False, False, False, False],
        [False, False, False, False, False, False],
        [False, False, False, False, False, False]])

In [12]:
data = (train_set[100])
for key in ["action", "state", "effect"]:
    data[key] = data[key].unsqueeze(0).expand(-1, 13, -1)

data["pad_mask"] = data["pad_mask"].unsqueeze(0).expand(-1, 13)

In [13]:
z, effect = model.forward(data,eval_mode=False)

Action performed on from 4 0 location, there really is an object in that location

In [36]:
train_set.action[100]

tensor([4, 0, 6, 5, 1, 0], dtype=torch.int32)

In [38]:
[get_obj_location(k) for k in train_set.state[100]]

[[7, 7],
 [5, 4],
 [4, 7],
 [4, 0],
 [1, 5],
 [2, 7],
 [7, 2],
 [6, 5],
 [3, 5],
 [0, 2],
 [6, 1],
 [0, 4],
 [0, 4]]

This action is not captured by the model output

In [14]:
effect > 1e-1

tensor([[[False, False, False, False, False,  True],
         [False, False, False, False, False, False],
         [False, False, False, False, False, False],
         [False, False, False, False, False,  True],
         [False, False, False, False, False, False],
         [False, False, False, False, False, False],
         [False, False, False, False, False, False],
         [False, False, False, False, False, False],
         [False, False, False, False, False, False],
         [False, False, False, False, False, False],
         [False, False, False, False, False, False],
         [False, False, False, False, False, False],
         [False, False, False, False, False, False]]], device='cuda:0')